In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e21/sample_submission.csv
/kaggle/input/dissolved-oxygen-prediction-in-river-water/train.csv
/kaggle/input/dissolved-oxygen-prediction-in-river-water/test.csv


# 😵‍💫 Improve a Fixed Model the Data-Centric Way! 😵‍💫¶ (Second Attempt)
Playground Series - Season 3, Episode 21

This is a very different type of challenge! For this challenge, your task is to improve a dataset that is being used to train a random forest model; in other words, your submission will be training data, not predictions. A random forest model will be trained on your submission, used to make predictions, and then those predictions will be used to generate your score.

The dataset for this competition is a synthetic dataset based off of the Dissolved oxygen prediction in river water dataset. You are free to use the original in any way that you find useful.

## 🔶 Look at the big Picture

### 🔸 **Frame the problem**
*   🟢 **Goal:** Is to fill the columns in the dataFrame such a way that the dataframe fed to the given model gives close to 0 RMSE. 🗒
*   🟢 **End Result:** multiple Columns to be predicted, changed such a way that models predict well 🗒
*   🟢 **Problem Type:** not exactly a regression. model has been already provided,
*   🟢 **Learning Type:** Supervised Learning
*   🟢 **Variate:** MultiVariate (need to predict multiple target with 1 label)
*   🟢 **Features:** single features

### 🔸 **Select performance measure**:
🟢 using RMSE 🗒.  This was given in the problem set.

### 🔸 **Assumptions**: `none`

---

## 🔶**Get the Data**

* 🟢 Add the sample_submission (as there are no train and test set in this problem) from the compition data
* 🟢 Read the sample_submission csv into a dataframe

In [2]:
#taking original training data from original data source

og_train_set = pd.read_csv('/kaggle/input/dissolved-oxygen-prediction-in-river-water/train.csv', index_col='Id')


#### 🔸 **Take Quick look**
* 🟢 .head() to see the index columns and quick look at the data 
* 🟢 .shape to see the size of training set
* 🟢 .info() to see the data types and missing values 
* 🟢 .value_counts() on categorical data to check if all the categories are equaly represented and to check the weightages.
* 🟢 .describe() to check mean and std deviation, min, max this gives a hint of how data is distributed and what is the max values, min values and is there any scaled data or not.

🚨 **Note:** Specially for this problem we are looking at the submision set so that we can have a clone of this dataframe structure and fill the data 

In [3]:
og_train_set.head()

,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,NH4_2,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
Id,,,,,,,,,,,,,,,,,,,,,
0,12.58,9.875,9.20,NaN,NaN,NaN,NaN,NaN,0.690,1.040,...,NaN,NaN,NaN,4.80,5.850,NaN,NaN,NaN,NaN,NaN
3,9.37,10.300,10.75,NaN,NaN,NaN,NaN,NaN,0.710,0.725,...,NaN,NaN,NaN,5.88,6.835,NaN,NaN,NaN,NaN,NaN
4,8.35,8.290,7.90,NaN,NaN,NaN,NaN,NaN,2.210,2.210,...,NaN,NaN,NaN,3.20,2.700,NaN,NaN,NaN,NaN,NaN
5,9.57,8.820,6.80,NaN,NaN,NaN,NaN,NaN,0.595,0.675,...,NaN,NaN,NaN,7.70,7.055,NaN,NaN,NaN,NaN,NaN
6,6.00,6.000,6.50,NaN,NaN,NaN,NaN,NaN,0.600,0.900,...,NaN,NaN,NaN,5.50,5.300,NaN,NaN,NaN,NaN,NaN


In [4]:
og_train_set.shape

(147, 36)

In [5]:
og_train_set.describe()

,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,NH4_2,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
count,147.000000,145.000000,145.000000,32.000000,31.000000,33.000000,37.000000,37.000000,145.000000,145.000000,...,33.000000,37.000000,37.000000,145.000000,145.000000,32.000000,31.000000,33.000000,37.000000,37.000000
mean,9.211497,9.168506,9.148126,10.002813,8.483226,6.095152,9.396892,7.598243,0.588168,0.660910,...,6.982515,3.373689,3.200135,4.972724,4.906379,4.665312,5.871935,8.678788,3.884054,3.957703
std,2.886614,4.188197,3.746898,2.667096,2.527239,3.279915,2.848153,3.409060,0.462262,0.591798,...,4.466642,2.823756,2.868225,2.044057,2.001381,2.057903,2.555560,5.299039,1.615542,1.916813
min,1.900000,2.200000,2.200000,4.660000,4.900000,0.000000,0.000000,0.000000,0.040000,0.020000,...,0.670000,0.100000,0.000000,0.900000,1.040000,2.040000,2.480000,2.880000,1.000000,1.600000
25%,7.375000,7.050000,7.200000,8.100000,6.865000,3.290000,8.800000,6.170000,0.230000,0.215000,...,4.100000,1.440000,1.610000,3.500000,3.400000,3.325000,4.445000,5.840000,2.900000,2.900000
50%,8.800000,8.700000,8.566667,9.522500,7.800000,5.800000,9.805000,7.150000,0.425000,0.440000,...,6.050000,2.650000,1.900000,4.800000,4.833333,4.150000,5.500000,7.300000,3.400000,3.500000
75%,10.850000,10.600000,10.500000,11.611250,9.232500,9.500000,10.440000,8.700000,0.800000,0.910000,...,8.550000,4.320000,3.800000,6.450000,6.200000,5.970000,6.400000,8.415000,4.800000,4.400000
max,19.100000,46.950000,40.900000,15.900000,15.400000,11.800000,13.700000,15.900000,2.400000,3.600000,...,20.050000,10.200000,13.300000,10.900000,11.000000,10.665000,14.790000,27.120000,8.540000,10.370000


In [6]:
og_train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147 entries, 0 to 214
Data columns (total 36 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   target  147 non-null    float64
 1   O2_1    145 non-null    float64
 2   O2_2    145 non-null    float64
 3   O2_3    32 non-null     float64
 4   O2_4    31 non-null     float64
 5   O2_5    33 non-null     float64
 6   O2_6    37 non-null     float64
 7   O2_7    37 non-null     float64
 8   NH4_1   145 non-null    float64
 9   NH4_2   145 non-null    float64
 10  NH4_3   32 non-null     float64
 11  NH4_4   31 non-null     float64
 12  NH4_5   33 non-null     float64
 13  NH4_6   37 non-null     float64
 14  NH4_7   37 non-null     float64
 15  NO2_1   145 non-null    float64
 16  NO2_2   145 non-null    float64
 17  NO2_3   32 non-null     float64
 18  NO2_4   31 non-null     float64
 19  NO2_5   33 non-null     float64
 20  NO2_6   37 non-null     float64
 21  NO2_7   37 non-null     float64
 22  NO